In [ ]:
import simplekml

In [ ]:
kml = simplekml.Kml()

# placemark = kml.newplacemarker()

# path = kml.newlinestring()
# path.coords = [(10,10), (20,20)]

point = kml.newpoint(name="Sample point", coords=[(49,9)])
print(kml.kml())

kml.save('Path.kml')

In [ ]:
import zipfile
import xml.etree.ElementTree as ET

# Open the KMZ file and extract the KML file
# with zipfile.ZipFile('Sample.kmz', 'r') as kmz:
    # kml_data = kmz.read('wpmz/template.kml')
    # kml_file = kmz.extract('wpmz/template.kml')
    # print(kml_data)

# Parse the KML file
# root = ET.fromstring(kml_data)


In [ ]:
import simplekml
kml = simplekml.Kml()
# schema = kml.newschema()
# schema.newsimplefield(name="FieldName", type='string')
# print(kml.document.allfeatures)
pnt = kml.newpoint(name="Point", coords=[(49,9,10)])
# pnt.extendeddata.newdata(name='new_field', value='new_value')
# kml.document.extendeddata(name="my_attribute", value="my_value")
# pnt.description = "Description"
# pnt.snippet.content = "snippet"
# pnt.snippet.maxlines = 1
print(kml.kml())

In [173]:
from xml.etree.ElementTree import Element, SubElement, tostring
import zipfile

class WMPLGenerator:
    
    def __init__(self):
        self.root = Element("kml")
        self.prefix = "wpml:"
        self.root.set("xmlns", "http://www.opengis.net/kml/2.2")
        self.root.set("xmlns:wpml", "http://www.dji.com/wpmz/1.0.2")
        # Step 1: Implement File Creation Information
        self.document = self.create_subelement(self.root, "Document")
        self.folder = self.create_subelement(self.document, "Folder")
        # Step 2: Setup Mission Configuration
        self._create_missionConfig()
        # Step 3: Setup A Folder for Waypoint Template
        self._create_folder()
        
    def build(self):
        return tostring(self.root)
    
    def save(self):
        # save the element to a file
        with open("waylines.wpml", "wb") as f:
            f.write(b"<?xml version='1.0' encoding='UTF-8'?>\n")
            f.write(self.build())
                        
    def create_element(self, tag, attrib={}):
        return Element(tag, attrib)
    
    def create_subelement(self, parent, tag, attrib={}):
        return SubElement(parent, tag, attrib)
    
    def _create_missionConfig(self):
        self.missionConfig = self.create_subelement(self.document, f"{self.prefix}missionConfig")
        
        self.flyToWaylineMode = self.create_subelement(self.missionConfig, f"{self.prefix}flyToWaylineMode")
        self.flyToWaylineMode.text = 'safely'
        
        self.finishAction = self.create_subelement(self.missionConfig, f"{self.prefix}finishAction")
        self.finishAction.text = 'goHome'
        
        self.exitOnRCLost = self.create_subelement(self.missionConfig, f"{self.prefix}exitOnRCLost")
        self.exitOnRCLost.text = 'executeLostAction'
        
        self.executeRCLostAction = self.create_subelement(self.missionConfig, f"{self.prefix}executeRCLostAction")
        self.executeRCLostAction.text = 'goBack'
        
        self.takeOffSecurityHeight = self.create_subelement(self.missionConfig, f"{self.prefix}takeOffSecurityHeight")
        self.takeOffSecurityHeight.text = '0'
        
        self.globalTransitionalSpeed = self.create_subelement(self.missionConfig, f"{self.prefix}globalTransitionalSpeed")
        self.globalTransitionalSpeed.text = '1'
        
        self.droneInfo = self.create_subelement(self.missionConfig, f"{self.prefix}droneInfo")
        
        self.droneEnumValue = self.create_subelement(self.droneInfo, f"{self.prefix}droneEnumValue")
        self.droneEnumValue.text = '60'
        
        self.droneSubEnumValue = self.create_subelement(self.droneInfo, f"{self.prefix}droneSubEnumValue")
        self.droneSubEnumValue.text = '0'
        
        # self.payloadInfo = self.create_subelement(self.missionConfig, f"{self.prefix}payloadInfo")
        
        # self.payloadEnumValue = self.create_subelement(self.payloadInfo, f"{self.prefix}payloadEnumValue")
        # self.payloadEnumValue.text = '42'

        # self.payloadSubEnumValue = self.create_subelement(self.payloadInfo, f"{self.prefix}payloadSubEnumValue")
        # self.payloadSubEnumValue.text = '0'

        # self.payloadPositionIndex = self.create_subelement(self.payloadInfo, f"{self.prefix}payloadPositionIndex")
        # self.payloadPositionIndex.text = '2'

    def _create_folder(self):
        self.templateId = self.create_subelement(self.folder, f"{self.prefix}templateId")
        self.templateId.text = '0'
                
        self.executeHeightMode = self.create_subelement(self.folder, f"{self.prefix}executeHeightMode")
        self.executeHeightMode.text = 'relativeToStartPoint'
        
        self.waylineId = self.create_subelement(self.folder, f"{self.prefix}waylineId")
        self.waylineId.text = '0'
        
        self.distance = self.create_subelement(self.folder, f"{self.prefix}distance")
        self.distance.text = '0'
        
        self.duration = self.create_subelement(self.folder, f"{self.prefix}duration")
        self.duration.text = '0'
        
        self.autoFlightSpeed = self.create_subelement(self.folder, f"{self.prefix}autoFlightSpeed")
        self.autoFlightSpeed.text = '1'        

    def create_placemark(self, idx=0, lat=0, lng=0, alt=10, vel=1):
        marker = self.create_subelement(self.folder, f"Placemark")

        point = self.create_subelement(marker, f"Point")
        coordinates = self.create_subelement(point, f"coordinates")
        coordinates.text = f"{lat}, {lng}"

        index = self.create_subelement(marker, f"{self.prefix}index")
        index.text = f'{idx}'

        executeHeight = self.create_subelement(marker, f"{self.prefix}executeHeight")
        executeHeight.text = f'{alt}'

        waypointSpeed = self.create_subelement(marker, f"{self.prefix}waypointSpeed")
        waypointSpeed.text = f'{vel}'

        waypointHeadingParam = self.create_subelement(marker, f"{self.prefix}waypointHeadingParam")
        
        waypointHeadingMode = self.create_subelement(waypointHeadingParam, f"{self.prefix}waypointHeadingMode")
        waypointHeadingMode.text = 'followWayline'

        waypointHeadingAngle = self.create_subelement(waypointHeadingParam, f"{self.prefix}waypointHeadingAngle")
        waypointHeadingAngle.text = '0'

        waypointPoiPoint = self.create_subelement(waypointHeadingParam, f"{self.prefix}waypointPoiPoint")
        waypointPoiPoint.text = '0.000000,0.000000,0.000000'

        waypointHeadingAngleEnable = self.create_subelement(waypointHeadingParam, f"{self.prefix}waypointHeadingAngleEnable")
        waypointHeadingAngleEnable.text = '0'

        waypointTurnParam = self.create_subelement(marker, f"{self.prefix}waypointTurnParam")

        waypointTurnMode = self.create_subelement(waypointTurnParam, f"{self.prefix}waypointTurnMode")
        waypointTurnMode.text = 'toPointAndStopWithDiscontinuityCurvature'

        waypointTurnDampingDist = self.create_subelement(waypointTurnParam, f"{self.prefix}waypointTurnDampingDist")
        waypointTurnDampingDist.text = f'0'

        useStraightLine = self.create_subelement(marker, f"{self.prefix}useStraightLine")
        useStraightLine.text = '0'
        
        return marker


class KMLGenerator:
    
    def __init__(self):
        self.root = Element("kml")
        self.prefix = "wpml:"
        self.root.set("xmlns", "http://www.opengis.net/kml/2.2")
        self.root.set("xmlns:wpml", "http://www.dji.com/wpmz/1.0.2")
        self.document = self.create_subelement(self.root, "Document")
        self.folder = self.create_subelement(self.document, "Folder")
        self._create_creation_time()
        self._create_update_time()
        self._create_missionConfig()
        self._create_folder()
        
    def build(self):
        return tostring(self.root)
    
    def save(self):
        # save the element to a file
        with open("template.kml", "wb") as f:
            f.write(b"<?xml version='1.0' encoding='UTF-8'?>\n")
            f.write(self.build())
                        
    def create_element(self, tag, attrib={}):
        return Element(tag, attrib)
    
    def create_subelement(self, parent, tag, attrib={}):
        return SubElement(parent, tag, attrib)
    
    def _create_creation_time(self):
        self.create_time = self.create_subelement(self.document, f"{self.prefix}createTime")
        self.create_time.text = '1674832081590'

    def _create_update_time(self):
        self.update_time = self.create_subelement(self.document, f"{self.prefix}updateTime")
        self.update_time.text = '1674841358573'

    def _create_missionConfig(self):
        self.missionConfig = self.create_subelement(self.document, f"{self.prefix}missionConfig")
        
        self.flyToWaylineMode = self.create_subelement(self.missionConfig, f"{self.prefix}flyToWaylineMode")
        self.flyToWaylineMode.text = 'safely'
        
        self.finishAction = self.create_subelement(self.missionConfig, f"{self.prefix}finishAction")
        self.finishAction.text = 'goHome'
        
        self.exitOnRCLost = self.create_subelement(self.missionConfig, f"{self.prefix}exitOnRCLost")
        self.exitOnRCLost.text = 'executeLostAction'
        
        self.executeRCLostAction = self.create_subelement(self.missionConfig, f"{self.prefix}executeRCLostAction")
        self.executeRCLostAction.text = 'goBack'
        
        self.takeOffSecurityHeight = self.create_subelement(self.missionConfig, f"{self.prefix}takeOffSecurityHeight")
        self.takeOffSecurityHeight.text = '0'
        
        self.globalTransitionalSpeed = self.create_subelement(self.missionConfig, f"{self.prefix}globalTransitionalSpeed")
        self.globalTransitionalSpeed.text = '1'
        
        self.droneInfo = self.create_subelement(self.missionConfig, f"{self.prefix}droneInfo")
        
        self.droneEnumValue = self.create_subelement(self.droneInfo, f"{self.prefix}droneEnumValue")
        self.droneEnumValue.text = '60'
        
        self.droneSubEnumValue = self.create_subelement(self.droneInfo, f"{self.prefix}droneSubEnumValue")
        self.droneSubEnumValue.text = '0'
        
        # self.payloadInfo = self.create_subelement(self.missionConfig, f"{self.prefix}payloadInfo")
        
        # self.payloadEnumValue = self.create_subelement(self.payloadInfo, f"{self.prefix}payloadEnumValue")
        # self.payloadEnumValue.text = '42'

        # self.payloadSubEnumValue = self.create_subelement(self.payloadInfo, f"{self.prefix}payloadSubEnumValue")
        # self.payloadSubEnumValue.text = '0'

        # self.payloadPositionIndex = self.create_subelement(self.payloadInfo, f"{self.prefix}payloadPositionIndex")
        # self.payloadPositionIndex.text = '2'

    def _create_folder(self):
        self.templateType = self.create_subelement(self.folder, f"{self.prefix}templateType")
        self.templateType.text = 'waypoint'
        
        self.templateId = self.create_subelement(self.folder, f"{self.prefix}templateId")
        self.templateId.text = '0'
        
        self.waylineCoordinateSysParam = self.create_subelement(self.folder, f"{self.prefix}waylineCoordinateSysParam")
        
        self.coordinateMode = self.create_subelement(self.waylineCoordinateSysParam, f"{self.prefix}coordinateMode")
        self.coordinateMode.text = 'WGS84'
        
        self.heightMode = self.create_subelement(self.waylineCoordinateSysParam, f"{self.prefix}heightMode")
        self.heightMode.text = 'relativeToStartPoint'
        
        self.positioningType = self.create_subelement(self.waylineCoordinateSysParam, f"{self.prefix}positioningType")
        self.positioningType.text = 'GPS'
        
        self.autoFlightSpeed = self.create_subelement(self.folder, f"{self.prefix}autoFlightSpeed")
        self.autoFlightSpeed.text = '1'
        
        self.globalHeight = self.create_subelement(self.folder, f"{self.prefix}globalHeight")
        self.globalHeight.text = '10'
        
        self.caliFlightEnable = self.create_subelement(self.folder, f"{self.prefix}caliFlightEnable")
        self.globalHeight.text = '0'
        
        self.gimbalPitchMode = self.create_subelement(self.folder, f"{self.prefix}gimbalPitchMode")
        self.gimbalPitchMode.text = 'manual'
        
        self.globalWaypointHeadingParam = self.create_subelement(self.folder, f"{self.prefix}globalWaypointHeadingParam")
        
        self.waypointHeadingMode = self.create_subelement(self.globalWaypointHeadingParam, f"{self.prefix}waypointHeadingMode")
        self.waypointHeadingMode.text = 'followWayline'
        
        self.waypointHeadingAngle = self.create_subelement(self.globalWaypointHeadingParam, f"{self.prefix}waypointHeadingAngle")
        self.waypointHeadingAngle.text = '0'
        
        self.waypointPoiPoint = self.create_subelement(self.globalWaypointHeadingParam, f"{self.prefix}waypointPoiPoint")
        self.waypointPoiPoint.text = '0.000000,0.000000,0.000000'
        
        self.globalWaypointTurnMode = self.create_subelement(self.folder, f"{self.prefix}globalWaypointTurnMode")
        self.globalWaypointTurnMode.text = 'coordinateTurn'
        
        self.globalUseStraightLine = self.create_subelement(self.folder, f"{self.prefix}globalUseStraightLine")
        self.globalUseStraightLine.text = '1'
        
        # self.payloadParam = self.create_subelement(self.folder, f"{self.prefix}payloadParam")
        
        # self.payloadPositionIndex = self.create_subelement(self.payloadParam, f"{self.prefix}payloadPositionIndex")
        # self.payloadPositionIndex.text = '0'      
        
        # self.meteringMode = self.create_subelement(self.payloadParam, f"{self.prefix}meteringMode")
        # self.meteringMode.text = 'average'      
        
        # self.dewarpingEnable = self.create_subelement(self.payloadParam, f"{self.prefix}dewarpingEnable")
        # self.dewarpingEnable.text = '0'      
        
        # self.returnMode = self.create_subelement(self.payloadParam, f"{self.prefix}returnMode")
        # self.returnMode.text = 'singleReturnStrongest'      
        
        # self.samplingRate = self.create_subelement(self.payloadParam, f"{self.prefix}samplingRate")
        # self.samplingRate.text = '240000'      
        
        # self.scanningMode = self.create_subelement(self.payloadParam, f"{self.prefix}scanningMode")
        # self.scanningMode.text = 'nonRepetitive'      
        
        # self.modelColoringEnable = self.create_subelement(self.payloadParam, f"{self.prefix}modelColoringEnable")
        # self.modelColoringEnable.text = '0'      
        
    def create_placemark(self, idx=0, lat=0, lng=0, alt=10, vel=1):
        marker = self.create_subelement(self.folder, f"Placemark")

        point = self.create_subelement(marker, f"Point")
        coordinates = self.create_subelement(point, f"coordinates")
        coordinates.text = f"{lat}, {lng}"

        index = self.create_subelement(marker, f"{self.prefix}index")
        index.text = f'{idx}'

        ellipsoidHeight = self.create_subelement(marker, f"{self.prefix}ellipsoidHeight")
        ellipsoidHeight.text = '100'

        height = self.create_subelement(marker, f"{self.prefix}height")
        height.text = f'{alt}'

        useGlobalHeight = self.create_subelement(marker, f"{self.prefix}useGlobalHeight")
        useGlobalHeight.text = '1'

        useGlobalSpeed = self.create_subelement(marker, f"{self.prefix}useGlobalSpeed")
        useGlobalSpeed.text = f'{vel}'

        useGlobalHeadingParam = self.create_subelement(marker, f"{self.prefix}useGlobalHeadingParam")
        useGlobalHeadingParam.text = '1'

        useGlobalTurnParam = self.create_subelement(marker, f"{self.prefix}useGlobalTurnParam")
        useGlobalTurnParam.text = '1'

        useStraightLine = self.create_subelement(marker, f"{self.prefix}useStraightLine")
        useStraightLine.text = '0'
        
        return marker


In [174]:
kml = KMLGenerator()
wmpl = WMPLGenerator()

kml.create_placemark(0, 9.97151628837884,49.7871353125553, 10, 1)
# kml.create_placemark(0, 10.4200331881162,51.8908883032862, 10, 1)
# kml.create_placemark(1, 10.4201295207333,51.8908958982527, 10, 1)
# kml.create_placemark(3, 10.4201295206877,51.8909724684922, 10, 1)
# kml.create_placemark(4, 10.4202495972781,51.8909707937753, 10, 1)
# kml.create_placemark(5, 10.4202862308041,51.8908966872905, 10, 1)
kml.save()

wmpl.create_placemark(0, 9.97151628837884,49.7871353125553, 10, 1)
# wmpl.create_placemark(0, 10.4200331881162,51.8908883032862, 10, 1)
# wmpl.create_placemark(1, 10.4201295207333,51.8908958982527, 10, 1)
# wmpl.create_placemark(3, 10.4201295206877,51.8909724684922, 10, 1)
# wmpl.create_placemark(4, 10.4202495972781,51.8909707937753, 10, 1)
# wmpl.create_placemark(5, 10.4202862308041,51.8908966872905, 10, 1)
wmpl.save()


with zipfile.ZipFile("Mission.kmz", "w") as kmz:
    # kmz.write(filename="", arcname="wpmz/")
    kmz.write(filename="template.kml", arcname="wpmz/template.kml")
    kmz.write(filename="waylines.wpml", arcname="wpmz/waylines.wpml")
